# Black Body Radiation Color Map

This iPython notebook contains the script required to derive a nice black body radation color map. The map is loosely based on the percieved colors of radiation from heated, opaque, non-reflecting bodies. The colors are adjusted such that the luminance is perceptually linear (according to the CIELAB color space).

This code relies on the [python-colormath](http://python-colormath.readthedocs.org/en/latest/index.html) module. See [its documentation](http://python-colormath.readthedocs.org/en/latest/index.html) for information such as installation instructions. (It can be installed with either pip or macports.)

In [310]:
from colormath.color_objects import *
from colormath.color_conversions import convert_color

Mostly because it's habit, I am also using [pandas](http://pandas.pydata.org/) dataframes to organize the data. (Pandas can be installed with macports.)

In [311]:
import pandas
import numpy

We will also be using [toyplot](https://toyplot.readthedocs.org) for making visuals. See its documentation for installation instructions.

In [312]:
import toyplot
import toyplot.pdf

The black body radiation color map goes from black to red to orange to yellow to white. Create color objects for these 5 control points.

In [313]:
color_rgb = [(0,0,0), (178,34,34), (227,105,5), (238,210,20), (255, 255, 255)]
data = pandas.DataFrame({'RGB': color_rgb})
data

,RGB
0,"(0, 0, 0)"
1,"(178, 34, 34)"
2,"(227, 105, 5)"
3,"(238, 210, 20)"
4,"(255, 255, 255)"


In [314]:
data['rgb_values'] = data['RGB'].apply(lambda rgb: sRGBColor(rgb[0], rgb[1], rgb[2],
                                                             is_upscaled=True))

Convert the rgb values to LAB values and get the luminance out of each of them to define the scalar (on a scale from 0 to 1) of each color.

In [315]:
data['lab_values'] = data['rgb_values'].apply(lambda rgb: convert_color(rgb, LabColor))
data['scalar'] = data['lab_values'].apply(lambda lab: lab.lab_l/100.0)
data

,RGB,rgb_values,lab_values,scalar
0,"(0, 0, 0)",sRGBColor (rgb_r:0.0000 rgb_g:0.0000 rgb_b:0.0...,LabColor (lab_l:0.0000 lab_a:0.0000 lab_b:0.0000),0.000000
1,"(178, 34, 34)",sRGBColor (rgb_r:0.6980 rgb_g:0.1333 rgb_b:0.1...,LabColor (lab_l:39.1167 lab_a:55.9150 lab_b:37...,0.391167
2,"(227, 105, 5)",sRGBColor (rgb_r:0.8902 rgb_g:0.4118 rgb_b:0.0...,LabColor (lab_l:58.4605 lab_a:43.3378 lab_b:65...,0.584605
3,"(238, 210, 20)",sRGBColor (rgb_r:0.9333 rgb_g:0.8235 rgb_b:0.0...,LabColor (lab_l:84.1336 lab_a:-6.4583 lab_b:82...,0.841336
4,"(255, 255, 255)",sRGBColor (rgb_r:1.0000 rgb_g:1.0000 rgb_b:1.0...,LabColor (lab_l:100.0000 lab_a:-0.0005 lab_b:-...,1.000000


Make a summary table of just the control points.

In [316]:
black_body_short = pandas.DataFrame(data, columns=['scalar', 'RGB'])
black_body_short

,scalar,RGB
0,0.000000,"(0, 0, 0)"
1,0.391167,"(178, 34, 34)"
2,0.584605,"(227, 105, 5)"
3,0.841336,"(238, 210, 20)"
4,1.000000,"(255, 255, 255)"


Make a function that will take a scalar value (in the range of 0 and 1) and return the appropriate RGB triple.

In [317]:
def black_body_lookup_upscaled(x):
    if x < 0:
        return (0, 0, 0)
    for index in xrange(0, 4):
        low_scalar = data['scalar'][index]
        high_scalar = data['scalar'][index+1]
        if (x > high_scalar):
            continue
        low_lab = data['lab_values'][index]
        high_lab = data['lab_values'][index+1]
        interp = (x-low_scalar)/(high_scalar-low_scalar)
        mid_lab = LabColor(interp*(high_lab.lab_l-low_lab.lab_l) + low_lab.lab_l,
                           interp*(high_lab.lab_a-low_lab.lab_a) + low_lab.lab_a,
                           interp*(high_lab.lab_b-low_lab.lab_b) + low_lab.lab_b)
        return convert_color(mid_lab, sRGBColor).get_upscaled_value_tuple()
    return (255, 255, 255)

def black_body_lookup(x):
    if x < 0:
        return (0.0, 0.0, 0.0)
    for index in xrange(0, 4):
        low_scalar = data['scalar'][index]
        high_scalar = data['scalar'][index+1]
        if (x > high_scalar):
            continue
        low_lab = data['lab_values'][index]
        high_lab = data['lab_values'][index+1]
        interp = (x-low_scalar)/(high_scalar-low_scalar)
        mid_lab = LabColor(interp*(high_lab.lab_l-low_lab.lab_l) + low_lab.lab_l,
                           interp*(high_lab.lab_a-low_lab.lab_a) + low_lab.lab_a,
                           interp*(high_lab.lab_b-low_lab.lab_b) + low_lab.lab_b)
        return convert_color(mid_lab, sRGBColor).get_value_tuple()
    return (1.0, 1.0, 1.0)

Make a long table of colors. This is a very high resolution table of colors that can be easily trimmed down with regular sampling.

In [318]:
black_body_long = pandas.DataFrame({'scalar': numpy.linspace(0.0, 1.0, num=1024)})
black_body_long['RGB'] = black_body_long['scalar'].apply(black_body_lookup_upscaled)
black_body_long['sRGB'] = black_body_long['scalar'].apply(black_body_lookup)

In [319]:
def unzip_rgb_triple(dataframe, column='RGB'):
    '''Given a dataframe and the name of a column holding an RGB triplet,
    this function creates new separate columns for the R, G, and B values
    with the same name as the original with '_r', '_g', and '_b' appended.'''
    # Creates a data frame with separate columns for the triples in the given column
    unzipped_rgb = pandas.DataFrame(dataframe[column].values.tolist(),
                                    columns=['r', 'g', 'b'])
    # Add the columns to the original data frame
    dataframe[column + '_r'] = unzipped_rgb['r']
    dataframe[column + '_g'] = unzipped_rgb['g']
    dataframe[column + '_b'] = unzipped_rgb['b']

unzip_rgb_triple(black_body_short, 'RGB')
unzip_rgb_triple(black_body_long, 'RGB')
unzip_rgb_triple(black_body_long, 'sRGB')

Check to make sure that all the colors are actually valid. The answer to this sum should be 0 if all the values are valid.

In [320]:
invalid = ((black_body_long['sRGB_r'] < 0) | (black_body_long['sRGB_r'] > 1) |
           (black_body_long['sRGB_g'] < 0) | (black_body_long['sRGB_g'] > 1) |
           (black_body_long['sRGB_b'] < 0) | (black_body_long['sRGB_b'] > 1))
num_bad_values = invalid.sum()
if num_bad_values > 0:
    raise ValueError, 'Found %d invalid colors!!!!' % num_bad_values

In [321]:
black_body_palette = toyplot.color.Palette(colors=black_body_long['sRGB'].values)
black_body_map = toyplot.color.LinearMap(palette=black_body_palette)
#black_body_map

In [329]:
canvas = toyplot.Canvas(width=512, height=200)
axes = canvas.axes()
axes.scatterplot(black_body_short['scalar'], numpy.zeros(black_body_short['scalar'].shape),
                 marker='|', size=1000,
                 color='gray')
axes.scatterplot(black_body_long['scalar'], numpy.zeros(black_body_long['scalar'].shape),
                 marker='|', size=500,
                 color=black_body_map)
axes.y.show = False
axes.x.show = False

for index in xrange(0, len(black_body_short.index), 2):
    axes.text(black_body_short['scalar'][index], 0,
              '%1.2f' % black_body_short['scalar'][index],
              style={'alignment-baseline':'hanging',
                     'baseline-shift':'-150%'},
              color='black')
    axes.text(black_body_short['scalar'][index], 0,
              str(black_body_short['RGB'][index]),
              style={'alignment-baseline':'hanging',
                     'baseline-shift':'-250%'},
              color='black')

for index in xrange(1, len(black_body_short.index), 2):
    axes.text(black_body_short['scalar'][index], 0,
              '%1.2f' % black_body_short['scalar'][index],
              style={'alignment-baseline':'alphabetic',
                     'baseline-shift':'260%'},
              color='black')
    axes.text(black_body_short['scalar'][index], 0,
              str(black_body_short['RGB'][index]),
              style={'alignment-baseline':'alphabetic',
                     'baseline-shift':'160%'},
              color='black')

In [330]:
toyplot.pdf.render(canvas, 'BlackBodyMap.pdf')